In [1]:
#load packages
import psycopg2
import pandas as pd

In [2]:
#connect do db
conn = psycopg2.connect(database="db_sync", user="", password="")
print(conn.closed)
print(conn.get_dsn_parameters(), "\n")

0
{'user': 'lukas', 'channel_binding': 'prefer', 'dbname': 'db_sync', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [52]:
%%bash
ls

data
db_connect.py
distance.py
format_business.py
format_yelpdata.py
psql.py
restaurants2.png
restaurants_map.py
reviewstarshist.png
review_stats.py
spezialthema-Copy1.ipynb
spezialthema.ipynb
stations.py
station_weather_to_db.ipynb
verteilung_stationen.png
weather.py
yelp_business.ipynb


In [3]:
def sql(sql: str):
    return pd.read_sql(sql, conn)

In [4]:
# create function to print tables
def print_table(tname: str):
    return pd.read_sql('select * from {} limit 10;'.format(tname), conn)

In [5]:
def delete_entries_table(tname: str):
    with conn:
        with conn.cursor() as cur:
            cur.execute('delete from {};'.format(tname))

In [6]:
#create function to load data in db
def station_to_db():
    stations = pd.read_csv("/home/sync/weather/stations.csv", sep=';')
    with conn:
        with conn.cursor() as cur:
            columns = list(stations.columns)
            for i in range(len(stations)):
                values = [stations[column][i] for column in columns]
                cur.execute('insert into station(id, latitude, longitude, elevation, state, name) values(%s, %s, %s, %s, %s, %s);', values)


In [7]:
print_table('station')

,id,latitude,longitude,elevation,state,name


In [9]:
#load stations
stations = pd.read_csv("../weather/stations.csv", sep=';')
stations.head()

,id,latitude,longitude,elevation,state,name
0,CA001010066,48.8667,-123.2833,4.0,BC,ACTIVE PASS
1,CA001010235,48.4000,-123.4833,17.0,BC,ALBERT HEAD
2,CA001010595,48.5833,-123.5167,85.0,BC,BAMBERTON OCEAN CEMENT
3,CA001010720,48.5000,-124.0000,351.0,BC,BEAR CREEK
4,CA001010774,48.5000,-123.3500,61.0,BC,BEAVER LAKE


In [10]:
stations.shape

(75369, 6)

In [11]:
stations.dtypes

id            object
latitude     float64
longitude    float64
elevation    float64
state         object
name          object
dtype: object

In [12]:
#print longest station
stations['name'].map(lambda x: len(x)).max()

30

In [12]:
#change datatype in db 
'''with conn:
    with conn.cursor() as cur:
        cur.execute('ALTER TABLE station ALTER COLUMN name TYPE character(30);')'''

In [8]:
station_to_db()

In [9]:
print_table("station")

,id,latitude,longitude,elevation,state,name
0,CA001010066,48.8667,-123.2833,4.0,BC,ACTIVE PASS ...
1,CA001010235,48.4000,-123.4833,17.0,BC,ALBERT HEAD ...
2,CA001010595,48.5833,-123.5167,85.0,BC,BAMBERTON OCEAN CEMENT ...
3,CA001010720,48.5000,-124.0000,351.0,BC,BEAR CREEK ...
4,CA001010774,48.5000,-123.3500,61.0,BC,BEAVER LAKE ...
5,CA001010780,48.3333,-123.6333,12.0,BC,BECHER BAY ...
6,CA001010960,48.6000,-123.4667,38.0,BC,BRENTWOOD BAY 2 ...
7,CA001010961,48.5667,-123.4500,31.0,BC,BRENTWOOD CLARKE ROAD ...
8,CA001010965,48.5667,-123.4333,91.0,BC,BRENTWOOD W SAANICH RD ...
9,CA001011467,48.5833,-123.4167,53.0,BC,CENTRAL SAANICH VEYANESS ...


In [10]:
sql('select count(id) count_stations from station')

,count_stations
0,75369


In [16]:
weather = pd.read_csv("../weather/2004_tidy.csv", sep=';')
weather.head()

,id,date,TMIN,TMAX,PRCP,SNOW,SNWD,TAVG,AWND,ACSC,PSUN
0,CA0010114F6,2004-01-01,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,CA0010114F6,2004-01-02,NaN,NaN,60.0,50.0,0.0,NaN,NaN,NaN,NaN
2,CA0010114F6,2004-01-03,NaN,NaN,0.0,0.0,50.0,NaN,NaN,NaN,NaN
3,CA0010114F6,2004-01-04,NaN,NaN,0.0,0.0,40.0,NaN,NaN,NaN,NaN
4,CA0010114F6,2004-01-05,NaN,NaN,0.0,0.0,20.0,NaN,NaN,NaN,NaN


In [17]:
weather.shape

(4814045, 11)

In [18]:
weather.dtypes

id       object
date     object
TMIN    float64
TMAX    float64
PRCP    float64
SNOW    float64
SNWD    float64
TAVG    float64
AWND    float64
ACSC    float64
PSUN    float64
dtype: object

In [25]:
def weather_to_db():
    for i in range(2005,2022): #2022
        print(i)
        weather = pd.read_csv("../weather/" + str(i) + "_tidy.csv", sep=';')
        with conn:
            with conn.cursor() as cur:
                columns = list(weather.columns)
                for i in range(len(weather)):
                    values = [weather[column][i] for column in columns]
                    cur.execute('insert into weather(id, date, tmin, tmax, prcp, snow, snwd, tavg, awnd, acsc, psun) \
                                values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);', values)


In [20]:
print_table('weather')

,date,id,tmin,tmax,prcp,snow,snwd,acsc,tavg,psun,awnd


In [26]:
weather_to_db()

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [23]:
sql('select count(id) count_weather from weather')

,count_weather
0,4814045


In [24]:
print_table('weather')

,date,id,tmin,tmax,prcp,snow,snwd,acsc,tavg,psun,awnd
0,2004-01-01,CA0010114F6,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2004-01-02,CA0010114F6,NaN,NaN,60.0,50.0,0.0,NaN,NaN,NaN,NaN
2,2004-01-03,CA0010114F6,NaN,NaN,0.0,0.0,50.0,NaN,NaN,NaN,NaN
3,2004-01-04,CA0010114F6,NaN,NaN,0.0,0.0,40.0,NaN,NaN,NaN,NaN
4,2004-01-05,CA0010114F6,NaN,NaN,0.0,0.0,20.0,NaN,NaN,NaN,NaN
5,2004-01-06,CA0010114F6,NaN,NaN,234.0,130.0,10.0,NaN,NaN,NaN,NaN
6,2004-01-07,CA0010114F6,NaN,NaN,16.0,0.0,50.0,NaN,NaN,NaN,NaN
7,2004-01-08,CA0010114F6,NaN,NaN,12.0,0.0,0.0,NaN,NaN,NaN,NaN
8,2004-01-09,CA0010114F6,NaN,NaN,8.0,0.0,0.0,NaN,NaN,NaN,NaN
9,2004-01-10,CA0010114F6,NaN,NaN,8.0,0.0,0.0,NaN,NaN,NaN,NaN


In [49]:
df = sql('''SELECT s.id, state, AVG(tmin) avg_tmin, AVG(tmax) avg_tmax FROM station s JOIN weather w ON s.id = w.id \
WHERE state='CA' AND EXTRACT(MONTH FROM date)=12 GROUP BY s.id;''')
df.dropna()

,id,state,avg_tmin,avg_tmax
2,USC00040029,CA,-30.033333,91.633333
6,USC00040192,CA,100.645161,217.032258
8,USC00040232,CA,41.580645,118.129032
11,USC00040379,CA,19.967742,131.000000
12,USC00040383,CA,38.961538,132.807692
...,...,...,...,...
759,USW00093228,CA,65.870968,145.354839
760,USW00093230,CA,-75.774194,55.516129
761,USW00093241,CA,42.161290,128.451613
763,USW00093243,CA,23.451613,114.000000


In [39]:
sql('''SELECT w.date, AVG(tmin) avg_tmin, AVG(tmax) avg_tmax FROM station s JOIN weather w ON s.id = w.id \
WHERE state='CA' GROUP BY date;''')

,date,avg_tmin,avg_tmax
0,2004-01-01,NaN,NaN
1,2004-01-02,NaN,NaN
2,2004-01-03,NaN,NaN
3,2004-01-04,NaN,NaN
4,2004-01-05,NaN,NaN
...,...,...,...
361,2004-12-27,NaN,NaN
362,2004-12-28,NaN,NaN
363,2004-12-29,NaN,NaN
364,2004-12-30,NaN,NaN


In [ ]:
conn.close()